# Set up model and hyperparameters

In [1]:
from models.WaveNetVAE.WaveVae import WaveNetVAE
from models.WaveNetVAE.WVData import WVDataset
import torch
from torch.utils.data import DataLoader
import warnings
warnings.filterwarnings("ignore")

"""
Hyperparameters
"""

learning_rate = 0.00001
batchsize = 8
device='cuda'
input_size = (40, 112)
upsamples = [2, 2, 2, 2, 2, 2, 2, 2]
zsize = 32

WaveVAE = WaveNetVAE(input_size,
                     num_hiddens = 768,
                     upsamples = upsamples,
                     zsize = zsize)

WaveVAE.to(device)

VAEDataset = WVDataset(audio_path = "../ConvDenoiser_Old/clips",
                       length = 4096,
                       sample_rate = 24000,
                       max_files = 200,
                       hop_length = 128)

val_VAEDataset = WVDataset(audio_path = "../ConvDenoiser_Old/testdata",
                       length = 4096,
                       sample_rate = 24000,
                       max_files = 200,
                       hop_length = 128)

VAEDataloader = DataLoader(VAEDataset,
                           batch_size = batchsize,
                           shuffle = True)

val_VAEDataloader = DataLoader(val_VAEDataset,
                           batch_size = batchsize,
                           shuffle = False)

WaveNet Receptive Field:  4093
4096


Loading and preprocessing files to dataset.:   0%|          | 0/200 [00:00<?, ?it/s]

4096


Loading and preprocessing files to dataset.:   0%|          | 0/33 [00:00<?, ?it/s]

# Test tensor

In [8]:
import torchaudio
onehot, mfcc, target = next(iter(VAEDataloader))
onehot = onehot.to(device)
mfcc = mfcc.to(device)
target = target.to(device)

print("Trying tensors with sizes:")
print("Onehot size: ", onehot.size(), "| MFCC size: ", mfcc.size(), "| Target size: ", target.size())

output = WaveVAE(onehot, mfcc, True)
print("Tensors passed through model succesfully")
onehot = torch.nn.functional.sigmoid(onehot[0, :, -1])
onehotmax = torch.argmax(onehot)
print(onehotmax, target[0])


Trying tensors with sizes:
Onehot size:  torch.Size([8, 256, 4096]) | MFCC size:  torch.Size([8, 40, 33]) | Target size:  torch.Size([8])
Tensors passed through model succesfully
tensor(209, device='cuda:0') tensor(220, device='cuda:0')


# Start training

In [5]:
from models.WaveNetVAE.train import train
from torch.utils.tensorboard import SummaryWriter
import warnings
warnings.filterwarnings("ignore")
writer = SummaryWriter()

train(WaveVAE, VAEDataloader, val_VAEDataloader, 
      writer = writer, 
      learning_rate = learning_rate,
      epoch_amount = 100,
      logs_per_epoch = 5,
      kl_anneal = 0.01,
      max_kl = 0.5,
      device = device)


Training. Epoch: 0. Loss for step 1: n.v.t.:   0%|          | 0/1051 [00:00<?, ?it/s]

KeyboardInterrupt: 